# Views vs Tables in SQL (Snowflake)

## Learning Objectives
- Understand what a view is and how it differs from a table
- Learn how views are executed by the database
- Explore views as CTEs (Common Table Expressions)
- Understand data warehouse use cases for views
- Learn about materialized views and their benefits



## 1. What is a View?

A **view** is a virtual table that is based on the result of a SQL query. Unlike a physical table, a view does not store data itself. Instead, it stores the query definition and executes it every time the view is accessed.

### Key Characteristics:
- **Virtual**: No physical storage of data
- **Dynamic**: Always reflects current data from underlying tables
- **Query-based**: Defined by a SELECT statement
- **Read-only or Updatable**: Depending on the query complexity

### Why Use Views?
1. **Simplify complex queries**: Hide complexity from end users
2. **Security**: Restrict access to specific columns/rows
3. **Consistency**: Provide standardized data access
4. **Abstraction**: Change underlying structure without affecting users


In [ ]:
-- First, let's create some sample tables in Snowflake
-- Create a customers table
CREATE OR REPLACE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    city VARCHAR(50),
    country VARCHAR(50),
    registration_date DATE
);

-- Create an orders table
CREATE OR REPLACE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR(20)
);

-- Insert dummy data into customers table
INSERT INTO customers VALUES
(1, 'John', 'Doe', 'john.doe@email.com', 'New York', 'USA', '2023-01-15'),
(2, 'Jane', 'Smith', 'jane.smith@email.com', 'London', 'UK', '2023-02-20'),
(3, 'Bob', 'Johnson', 'bob.johnson@email.com', 'Toronto', 'Canada', '2023-03-10'),
(4, 'Alice', 'Williams', 'alice.williams@email.com', 'Sydney', 'Australia', '2023-04-05'),
(5, 'Charlie', 'Brown', 'charlie.brown@email.com', 'New York', 'USA', '2023-05-12');

-- Insert dummy data into orders table
INSERT INTO orders VALUES
(101, 1, '2023-06-01', 150.00, 'Completed'),
(102, 1, '2023-07-15', 250.50, 'Completed'),
(103, 2, '2023-06-20', 75.25, 'Pending'),
(104, 3, '2023-08-01', 320.00, 'Completed'),
(105, 4, '2023-08-10', 180.75, 'Completed'),
(106, 1, '2023-09-05', 95.50, 'Pending'),
(107, 5, '2023-09-12', 210.00, 'Completed');


In [ ]:
-- Example 1: Create a simple view
-- This view shows customer names and their email addresses
CREATE OR REPLACE VIEW customer_contacts AS
SELECT 
    customer_id,
    first_name || ' ' || last_name AS full_name,
    email,
    city,
    country
FROM customers;

-- Query the view (it works just like a table)
SELECT * FROM customer_contacts;


In [ ]:
-- Example 2: Create a view with joins and aggregations
-- This view shows customer order summary
CREATE OR REPLACE VIEW customer_order_summary AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.first_name, c.last_name, c.country;

-- Query the view
SELECT * FROM customer_order_summary
ORDER BY total_spent DESC;


## 2. View vs Table - Key Differences

| Aspect | Table | View |
|--------|-------|------|
| **Storage** | Physical storage of data | No physical storage (virtual) |
| **Data** | Contains actual data | Contains query definition |
| **Performance** | Fast (direct data access) | Slower (query executed each time) |
| **DML Operations** | INSERT, UPDATE, DELETE supported | Limited (only simple views) |
| **Storage Cost** | Consumes storage space | No storage cost |
| **Data Freshness** | Static until updated | Always current (dynamic) |
| **Indexing** | Can create indexes | Cannot create indexes directly |
| **Dependencies** | Independent | Depends on underlying tables |


In [ ]:
-- Demonstration: View vs Table

-- Create a table (physical storage)
CREATE OR REPLACE TABLE customer_ny_table AS
SELECT 
    customer_id,
    first_name || ' ' || last_name AS full_name,
    email,
    city
FROM customers
WHERE city = 'New York';

-- Create a view (virtual, no storage)
CREATE OR REPLACE VIEW customer_ny_view AS
SELECT 
    customer_id,
    first_name || ' ' || last_name AS full_name,
    email,
    city
FROM customers
WHERE city = 'New York';

-- Both return the same data initially
SELECT 'Table Results:' AS source, * FROM customer_ny_table;
SELECT 'View Results:' AS source, * FROM customer_ny_view;

-- Now add a new customer in New York
INSERT INTO customers VALUES
(6, 'David', 'Miller', 'david.miller@email.com', 'New York', 'USA', '2023-10-01');

-- Query again - see the difference!
SELECT 'Table Results (after insert):' AS source, * FROM customer_ny_table;
SELECT 'View Results (after insert):' AS source, * FROM customer_ny_view;

-- The table still has old data (needs to be refreshed)
-- The view automatically shows new data (dynamic)


## 3. Views as CTE (Common Table Expressions)

A **CTE (Common Table Expression)** is a temporary named result set that exists only for the duration of a single query. Views can be thought of as **persistent CTEs** that can be reused across multiple queries.

### CTE vs View:
- **CTE**: Temporary, exists only within the query scope
- **View**: Persistent, stored in database, can be reused
- **CTE**: Better for one-time complex queries
- **View**: Better for reusable query logic


In [ ]:
-- Example: Using CTE (temporary, query-scoped)
WITH customer_stats AS (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        COUNT(o.order_id) AS order_count,
        SUM(o.total_amount) AS total_spent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT 
    customer_name,
    order_count,
    total_spent,
    CASE 
        WHEN total_spent > 300 THEN 'High Value'
        WHEN total_spent > 150 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS customer_segment
FROM customer_stats
ORDER BY total_spent DESC;

-- The CTE above only exists for this query
-- If you want to reuse this logic, create a view instead


In [ ]:
-- Example: Converting CTE to View (persistent, reusable)
-- Create a view that can be used in multiple queries
CREATE OR REPLACE VIEW customer_segmentation AS
WITH customer_stats AS (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        COUNT(o.order_id) AS order_count,
        SUM(o.total_amount) AS total_spent
    FROM customers c
    LEFT JOIN orders o ON c.customer_id = o.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
)
SELECT 
    customer_id,
    customer_name,
    order_count,
    total_spent,
    CASE 
        WHEN total_spent > 300 THEN 'High Value'
        WHEN total_spent > 150 THEN 'Medium Value'
        ELSE 'Low Value'
    END AS customer_segment
FROM customer_stats;

-- Now you can use this view in multiple queries
SELECT * FROM customer_segmentation WHERE customer_segment = 'High Value';
SELECT * FROM customer_segmentation ORDER BY order_count DESC;


## 4. How Database Executes Views

Understanding how a database executes views is crucial for performance optimization.

### Execution Process:

1. **Query Parsing**: Database parses the view definition
2. **Query Merging**: The view's SELECT statement is merged with the query using the view
3. **Query Optimization**: The combined query is optimized
4. **Execution**: The optimized query is executed against underlying tables
5. **Result Return**: Results are returned to the user

### Important Points:
- Views are **not pre-executed** - they execute on-demand
- The view query is **merged** with your query (not executed separately)
- Performance depends on the underlying tables and indexes
- Views can be used in WHERE clauses, JOINs, etc.


In [ ]:
-- Example: Understanding view execution

-- Create a view
CREATE OR REPLACE VIEW active_customers AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(o.order_id) AS order_count
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'Completed'
GROUP BY c.customer_id, c.first_name, c.last_name, c.country
HAVING COUNT(o.order_id) >= 2;

-- When you query the view with a WHERE clause:
-- The database merges your WHERE with the view's query
SELECT customer_name, order_count 
FROM active_customers 
WHERE country = 'USA';

-- This is equivalent to:
SELECT customer_name, order_count
FROM (
    SELECT 
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        c.country,
        COUNT(o.order_id) AS order_count
    FROM customers c
    INNER JOIN orders o ON c.customer_id = o.customer_id
    WHERE o.status = 'Completed'
    GROUP BY c.customer_id, c.first_name, c.last_name, c.country
    HAVING COUNT(o.order_id) >= 2
) AS active_customers
WHERE country = 'USA';

-- The database optimizes this merged query before execution


In [ ]:
-- In Snowflake, you can see the query plan using EXPLAIN
-- This shows how the view query is merged and optimized
EXPLAIN 
SELECT customer_name, order_count 
FROM active_customers 
WHERE country = 'USA';


## 5. Data Warehouse Use Cases for Views

Views are particularly valuable in data warehouse environments:

### Common Use Cases:

1. **Data Security & Access Control**
   - Restrict sensitive columns
   - Row-level security
   - Role-based access

2. **Data Abstraction**
   - Hide complex joins and transformations
   - Provide business-friendly names
   - Standardize data access patterns

3. **Data Integration**
   - Combine data from multiple sources
   - Create unified business views
   - Handle schema differences

4. **Performance Optimization**
   - Pre-filter and pre-aggregate data
   - Reduce query complexity for end users
   - Optimize frequently used queries

5. **Business Logic Encapsulation**
   - Store calculated fields
   - Implement business rules
   - Maintain consistency across reports


In [ ]:
-- Use Case 1: Security - Hide sensitive data
-- Create a view that excludes sensitive email addresses
CREATE OR REPLACE VIEW public_customer_info AS
SELECT 
    customer_id,
    first_name || ' ' || last_name AS customer_name,
    city,
    country,
    registration_date
    -- Note: email is excluded for security
FROM customers;

-- Use Case 2: Business-friendly names and calculations
CREATE OR REPLACE VIEW sales_dashboard AS
SELECT 
    c.country,
    DATE_TRUNC('month', o.order_date) AS order_month,
    COUNT(DISTINCT o.order_id) AS total_orders,
    COUNT(DISTINCT c.customer_id) AS unique_customers,
    SUM(o.total_amount) AS revenue,
    AVG(o.total_amount) AS avg_order_value,
    SUM(CASE WHEN o.status = 'Completed' THEN o.total_amount ELSE 0 END) AS completed_revenue
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.country, DATE_TRUNC('month', o.order_date)
ORDER BY order_month DESC, revenue DESC;

-- Query the business-friendly view
SELECT * FROM sales_dashboard WHERE country = 'USA';


In [ ]:
-- Use Case 3: Row-level security (filtering by conditions)
-- Create a view that only shows customers from specific countries
CREATE OR REPLACE VIEW us_canada_customers AS
SELECT 
    customer_id,
    first_name || ' ' || last_name AS customer_name,
    city,
    country,
    email
FROM customers
WHERE country IN ('USA', 'Canada');

-- Use Case 4: Pre-aggregated metrics for faster reporting
CREATE OR REPLACE VIEW monthly_sales_summary AS
SELECT 
    DATE_TRUNC('month', order_date) AS month,
    COUNT(*) AS order_count,
    SUM(total_amount) AS total_revenue,
    AVG(total_amount) AS avg_order_amount,
    MIN(total_amount) AS min_order_amount,
    MAX(total_amount) AS max_order_amount,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM orders
WHERE status = 'Completed'
GROUP BY DATE_TRUNC('month', order_date)
ORDER BY month DESC;

-- This pre-aggregated view makes reporting much faster
SELECT * FROM monthly_sales_summary;


## 6. Materialized Views

A **materialized view** is a view that stores the query results physically, like a table. Unlike regular views, materialized views pre-compute and store the data, which significantly improves query performance.

### Regular View vs Materialized View:

| Aspect | Regular View | Materialized View |
|--------|--------------|-------------------|
| **Storage** | No physical storage | Stores data physically |
| **Performance** | Executes query each time | Fast (reads stored data) |
| **Data Freshness** | Always current | Needs refresh |
| **Storage Cost** | None | Consumes storage |
| **Refresh** | Automatic (always current) | Manual or scheduled |
| **Use Case** | Real-time data | Pre-aggregated/analytical data |

### When to Use Materialized Views:
- Complex aggregations that are expensive to compute
- Frequently accessed analytical queries
- Data that doesn't need to be real-time
- Performance-critical reporting


In [ ]:
-- Example: Creating a Materialized View in Snowflake
-- Note: Snowflake supports materialized views with automatic refresh

-- Create a materialized view for fast analytical queries
CREATE OR REPLACE MATERIALIZED VIEW customer_analytics_mv AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS lifetime_value,
    AVG(o.total_amount) AS avg_order_value,
    MIN(o.order_date) AS first_order_date,
    MAX(o.order_date) AS last_order_date,
    DATEDIFF('day', MIN(o.order_date), MAX(o.order_date)) AS customer_lifespan_days
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.status = 'Completed'
GROUP BY c.customer_id, c.first_name, c.last_name, c.country;

-- Query the materialized view (very fast!)
SELECT 
    country,
    COUNT(*) AS customer_count,
    AVG(lifetime_value) AS avg_lifetime_value,
    AVG(total_orders) AS avg_orders_per_customer
FROM customer_analytics_mv
GROUP BY country
ORDER BY avg_lifetime_value DESC;


In [ ]:
-- Refresh a materialized view (if needed)
-- In Snowflake, materialized views can auto-refresh, but you can also manually refresh
ALTER MATERIALIZED VIEW customer_analytics_mv REFRESH;

-- Compare performance: Regular View vs Materialized View
-- Regular view executes the query each time
SELECT * FROM customer_order_summary WHERE total_spent > 200;

-- Materialized view reads from stored data (much faster for large datasets)
SELECT * FROM customer_analytics_mv WHERE lifetime_value > 200;


## Key Takeaways

1. **Views are virtual tables** - They don't store data, only query definitions
2. **Views are dynamic** - Always reflect current data from underlying tables
3. **Views simplify complexity** - Hide complex joins and logic from end users
4. **Views provide security** - Control access to sensitive data
5. **Views execute on-demand** - Query is merged and optimized at runtime
6. **Materialized views store data** - Better performance but need refresh
7. **Choose wisely** - Use regular views for real-time data, materialized views for analytics


## Practice Problems

### Problem 1: Create a View for Active Customers
Create a view that shows customers who have placed at least 2 orders with a total value greater than $200. Include customer name, email, total orders, and total spent.


In [ ]:
-- Solution to Problem 1
CREATE OR REPLACE VIEW active_customers_view AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.email,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent
FROM customers c
INNER JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'Completed'
GROUP BY c.customer_id, c.first_name, c.last_name, c.email
HAVING COUNT(o.order_id) >= 2 AND SUM(o.total_amount) > 200;

-- Verify the view
SELECT * FROM active_customers_view;


### Problem 2: Create a View with Business Logic
Create a view that categorizes customers into segments:
- "Platinum": Total spent > $300
- "Gold": Total spent between $200 and $300
- "Silver": Total spent between $100 and $200
- "Bronze": Total spent < $100

Include customer name, country, total spent, and segment.


In [ ]:
-- Solution to Problem 2
CREATE OR REPLACE VIEW customer_segments_view AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COALESCE(SUM(o.total_amount), 0) AS total_spent,
    CASE 
        WHEN COALESCE(SUM(o.total_amount), 0) > 300 THEN 'Platinum'
        WHEN COALESCE(SUM(o.total_amount), 0) BETWEEN 200 AND 300 THEN 'Gold'
        WHEN COALESCE(SUM(o.total_amount), 0) BETWEEN 100 AND 200 THEN 'Silver'
        ELSE 'Bronze'
    END AS customer_segment
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.status = 'Completed'
GROUP BY c.customer_id, c.first_name, c.last_name, c.country;

-- Verify the view
SELECT * FROM customer_segments_view ORDER BY total_spent DESC;


### Problem 3: Create a View for Monthly Sales Report
Create a view that shows monthly sales statistics including:
- Month (YYYY-MM format)
- Total revenue
- Number of orders
- Average order value
- Number of unique customers
- Top customer (by revenue) for that month


In [ ]:
-- Solution to Problem 3
CREATE OR REPLACE VIEW monthly_sales_report AS
WITH monthly_customer_sales AS (
    SELECT 
        DATE_TRUNC('month', o.order_date) AS order_month,
        c.customer_id,
        c.first_name || ' ' || c.last_name AS customer_name,
        SUM(o.total_amount) AS customer_revenue
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    WHERE o.status = 'Completed'
    GROUP BY DATE_TRUNC('month', o.order_date), c.customer_id, c.first_name, c.last_name
),
monthly_stats AS (
    SELECT 
        DATE_TRUNC('month', o.order_date) AS order_month,
        SUM(o.total_amount) AS total_revenue,
        COUNT(*) AS order_count,
        AVG(o.total_amount) AS avg_order_value,
        COUNT(DISTINCT o.customer_id) AS unique_customers
    FROM orders o
    WHERE o.status = 'Completed'
    GROUP BY DATE_TRUNC('month', o.order_date)
),
top_customers AS (
    SELECT 
        order_month,
        customer_name,
        customer_revenue,
        ROW_NUMBER() OVER (PARTITION BY order_month ORDER BY customer_revenue DESC) AS rn
    FROM monthly_customer_sales
)
SELECT 
    TO_CHAR(ms.order_month, 'YYYY-MM') AS month,
    ms.total_revenue,
    ms.order_count,
    ms.avg_order_value,
    ms.unique_customers,
    tc.customer_name AS top_customer,
    tc.customer_revenue AS top_customer_revenue
FROM monthly_stats ms
LEFT JOIN top_customers tc ON ms.order_month = tc.order_month AND tc.rn = 1
ORDER BY ms.order_month DESC;

-- Verify the view
SELECT * FROM monthly_sales_report;


### Problem 4: Create a Materialized View for Performance
Create a materialized view that pre-aggregates customer order statistics by country. Include:
- Country
- Total customers
- Total orders
- Total revenue
- Average revenue per customer

This should be optimized for fast analytical queries.


In [ ]:
-- Solution to Problem 4
CREATE OR REPLACE MATERIALIZED VIEW country_sales_analytics_mv AS
SELECT 
    c.country,
    COUNT(DISTINCT c.customer_id) AS total_customers,
    COUNT(DISTINCT o.order_id) AS total_orders,
    COALESCE(SUM(o.total_amount), 0) AS total_revenue,
    COALESCE(SUM(o.total_amount), 0) / NULLIF(COUNT(DISTINCT c.customer_id), 0) AS avg_revenue_per_customer,
    COALESCE(AVG(o.total_amount), 0) AS avg_order_value
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.status = 'Completed'
GROUP BY c.country;

-- Query the materialized view (fast!)
SELECT * FROM country_sales_analytics_mv ORDER BY total_revenue DESC;


### Problem 5: View with Security Filtering
Create a view that shows order details but excludes pending orders and hides customer email addresses. This view should be suitable for sharing with external stakeholders who shouldn't see sensitive information.


In [ ]:
-- Solution to Problem 5
CREATE OR REPLACE VIEW public_order_summary AS
SELECT 
    o.order_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.city,
    c.country,
    -- Email is excluded for security
    o.order_date,
    o.total_amount,
    o.status
FROM orders o
INNER JOIN customers c ON o.customer_id = c.customer_id
WHERE o.status = 'Completed'  -- Only show completed orders
ORDER BY o.order_date DESC;

-- Verify the view (no emails, no pending orders)
SELECT * FROM public_order_summary;


### Problem 6: Nested Views
Create a base view showing customer order details, then create another view on top of it that shows only high-value customers (total spent > $250). This demonstrates how views can be built on top of other views.


In [ ]:
-- Solution to Problem 6: Nested Views
-- Step 1: Create base view with customer order details
CREATE OR REPLACE VIEW customer_order_details AS
SELECT 
    c.customer_id,
    c.first_name || ' ' || c.last_name AS customer_name,
    c.country,
    COUNT(o.order_id) AS total_orders,
    SUM(o.total_amount) AS total_spent,
    AVG(o.total_amount) AS avg_order_value,
    MIN(o.order_date) AS first_order_date,
    MAX(o.order_date) AS last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id AND o.status = 'Completed'
GROUP BY c.customer_id, c.first_name, c.last_name, c.country;

-- Step 2: Create a view on top of the base view (nested view)
CREATE OR REPLACE VIEW high_value_customers AS
SELECT 
    customer_id,
    customer_name,
    country,
    total_orders,
    total_spent,
    avg_order_value,
    first_order_date,
    last_order_date
FROM customer_order_details
WHERE total_spent > 250
ORDER BY total_spent DESC;

-- Query the nested view
SELECT * FROM high_value_customers;


## Additional Notes

### View Management in Snowflake

- **View Metadata**: Use `SHOW VIEWS` to list all views
- **View Definition**: Use `SHOW VIEWS LIKE 'view_name'` or query `INFORMATION_SCHEMA.VIEWS`
- **Drop View**: `DROP VIEW view_name`
- **Alter View**: `ALTER VIEW view_name RENAME TO new_name`

### Best Practices

1. **Naming Convention**: Use clear, descriptive names (e.g., `customer_order_summary_view`)
2. **Documentation**: Add comments to views explaining their purpose
3. **Performance**: Monitor view performance; consider materialized views for heavy aggregations
4. **Security**: Use views to implement row-level and column-level security
5. **Maintenance**: Regularly review and update views as business requirements change
6. **Testing**: Test views after changes to underlying tables


In [ ]:
-- View management commands in Snowflake

-- List all views in the current schema
SHOW VIEWS;

-- Show materialized views
SHOW MATERIALIZED VIEWS;

-- Get view definition
SHOW VIEWS LIKE 'customer_order_summary';

-- Query view metadata from information schema
SELECT 
    table_catalog,
    table_schema,
    table_name,
    view_definition
FROM INFORMATION_SCHEMA.VIEWS
WHERE table_schema = CURRENT_SCHEMA()
ORDER BY table_name;

-- Drop a view
-- DROP VIEW IF EXISTS view_name;
